In [16]:
import numpy as np
import soundfile
from IPython.display import Audio

In [17]:
# FBCFNOMOD
# This function creates a feedback comb filter by processing an individual
# input sample and updating a delay buffer used in a loop to index each
# sample in a signal. This implementation does not use fractional delay.
# Therefore, the delay time cannot be modulated.
#
# Input Variables
#   n: current sample number of the input sample
#   delay: samples of delay
#   fbGain: feedback gain (linear scale)
#
# See also FBCF

def fbcfNoMod(x, buffer, n, delay, fbGain):
    # Determine indexes for circular buffer
    M = len(buffer)
    indexC = n % M # Current index
    indexD = int((n - delay)) % M # Delay index

    out = buffer[indexD]

    # Store the current output in appropriate index
    buffer[indexC] = x + fbGain * buffer[indexD]

    return out, buffer

In [18]:
# FBCF
# This function creates a feedback comb filter by processing an
# individual input sample and updating a delay buffer used in a loop
# to index each sample in a signal. Fractional delay is implemented
# to make it possible to modulate the delay time.
#
# Input Variables
#   n: current sample number of the input sample
#   delay: samples of delay
#   fbGain: feedback gain (linear scale)
#   amp: amplitude of LFO modulation
#   rate: frequency of LFO modulation
#
# See also FBCFNOMOD

def fbcf(x, buffer, Fs, n, delay, fbGain, amp, rate):
    # Calculate time in seconds for the current sample
    t = n/Fs
    fracDelay = amp * np.sin(2 * np.pi * rate * t)
    intDelay = int(np.floor(fracDelay))
    frac = fracDelay - intDelay

    # Determine indexes for circular buffer
    M = len(buffer)
    indexC = int(n % M)  # Current index
    indexD = int((n - delay + intDelay) % M)  # Delay index
    indexF = int((n - delay + intDelay + 1) % M) # Fractional index

    out = (1 - frac) * buffer[indexD] + frac * buffer[indexF]

    # Store the current output in appropriate index
    buffer[indexC] = x + fbGain * out

    return out, buffer

In [19]:
# FBCFEXAMPLE
# This script uses a feedback comb filter (FBCF) function applied
# to an acoustic guitar recording.
#
# See also FBCFNOMOD, FBCF

x, Fs = soundfile.read('AcGtr.wav')

maxDelay = int(np.ceil(0.05 * Fs)) # maximum delay of 50ms
buffer = np.zeros(maxDelay) # initialize delay buffer

d = 0.04 * Fs # 40ms of delay
g = -0.7 # feedback gain value

rate = 0.6 # Hz (frequency of LFO)
amp = 6 # Range of +/- 6 samples for delay

# Initialize output signal
N = len(x)
out = np.zeros(N)

for n in range(N):

    # Uncomment to use fbcfNoMod function
    # out[n], buffer = fbcfNoMod(x[n], buffer, n, d, g)

    # Use fbcf function
    out[n], buffer = fbcf(x[n], buffer, Fs, n, d, g, amp, rate)

Audio(out, rate=Fs)

In [21]:
# FBCFSERIESEXAMPLE
# This script uses series feedback comb filter (FBCF) functions, applied to
# an acoustic guitar recording.
#
# See also FBCFPARALLELEXAMPLE

x, Fs = soundfile.read('AcGtr.wav')

maxDelay = int(np.ceil(0.07 * Fs))  # max delay of 70ms
buffer1 = np.zeros(maxDelay)
buffer2 = np.zeros(maxDelay)

d1 = np.fix(0.042 * Fs) # 42ms of delay
g1 = 0.5
d2 = np.fix(0.053 * Fs) # 53ms of delay
g2 = -0.5

rate1 = 0.6 # Hz (frequency of LFO)
amp1 = 6 # Range of +/- 6 samples for delay
rate2 = 0.5 # Hz (frequency of LFO)
amp2 = 8 # Range of +/- 8 samples for delay

# Initialize output signal
N = len(x)
out = np.zeros(N)

for n in range(N):

    # Two series FBCFs
    w, buffer1 = fbcf(x[n], buffer1, Fs, n, d1, g1, amp1, rate1)

    # The output 'w' of the first FBCF is used as the input
    # of the second FBCF
    out[n], buffer2 = fbcf(w, buffer2, Fs, n, d2, g2, amp2, rate2)

Audio(out, rate=Fs)

In [22]:
# FBCFPARALLELEXAMPLE
# This script uses parallel feedback comb filter (FBCF) functions
# applied to an acoustic guitar recording.
#
# See also FBCFSERIESEXAMPLE

x, Fs = soundfile.read('AcGtr.wav')

maxDelay = int(np.ceil(0.07 * Fs)) # max delay of 70ms
buffer1 = np.zeros(maxDelay)
buffer2 = np.zeros(maxDelay)

d1 = np.fix(0.047 * Fs) # 47ms of delay
g1 = 0.5
d2 = np.fix(0.053 * Fs) # 53ms of delay
g2 = -0.5

rate1 = 0.6 # Hz (frequency of LFO)
amp1 = 6 # Range of +/- 6 samples for delay
rate2 = 0.5 # Hz (frequency of LFO)
amp2 = 8 # Range of +/- 8 samples for delay

# Initialize output signal
N = len(x)
out = np.zeros(N)

for n in range(N):
    # Two parallel FBCFs
    w1, buffer1 = fbcf(x[n], buffer1, Fs, n, d1, g1, amp1, rate1)

    # Both FBCF receive 'x' to create parallel processing
    w2, buffer2 = fbcf(x[n], buffer2, Fs, n, d2, g2, amp2, rate2)

    # The output of each FBCF is summed together to complete parallel processing
    out[n] = w1 + w2

Audio(out, rate=Fs)

In [28]:
# APF
# This function creates an all-pass filter by processing an individual
# input sample and updating a delay buffer used in a loop to index each
# sample in a signal.
#
# Input Variables
#   n: current sample number of the input signal
#   delay: samples of delay
#   gain: feedback gain (linear scale)
#   amp: amplitude of LFO modulation
#   rate: frequency of LFO modulation

def apf(x, buffer, Fs, n, delay, gain, amp, rate):
    # Calculate time in seconds for the current sample
    t = n/Fs
    fracDelay = amp * np.sin(2 * np.pi * rate * t)
    intDelay = int(np.floor(fracDelay))
    frac = fracDelay - intDelay

    # Determine indexes for circular buffer
    M = len(buffer)
    indexC = int(n % M) # Current index
    indexD = int((n-delay+intDelay) % M) # Delay index
    indexF = int((n-delay+intDelay+1) % M) # Fractional index

    # Temp variable for output of delay buffer
    w = (1 - frac) * buffer[indexD] + frac * buffer[indexF]

    # Temp variable used for the node after the input sum
    v = x + (-gain * w)

    # Summation at output
    out = (gain * v) + w

    # Store the current input to delay buffer
    buffer[indexC] = v

    return out, buffer

In [32]:
# APFEXAMPLE
# This script uses an all-pass filter function applied to an acoustic
# guitar recording.
#
# See also APF

x, Fs = soundfile.read('AcGtr.wav')

maxDelay = int(np.ceil(0.05 * Fs)) # maximum delay of 50ms
buffer = np.zeros(maxDelay)

d = np.ceil(0.042 * Fs) # 42ms of delay
g = 0.9

rate = 0.9 # Hz (frequency of LFO)
amp = 6 # Range of +/- 6 samples for delay

# Initialize output signal
N = len(x)
out = np.zeros(N)

for n in range(N):
    # Use apf function
    out[n], buffer = apf(x[n], buffer, Fs, n, d, g, amp, rate)

Audio(out, rate=Fs)

In [33]:
# SCHROEDERREVERB
# This script implements the Schroeder reverb algorithm by using feedback
# comb filters (fbcf) and all-pass filters (apf).
#
# See also FBCF, APF

x, Fs = soundfile.read('AcGtr.wav')

# Max delay of 70ms
maxDelay = int(np.ceil(0.07 * Fs))
# Initialize all buffers (there are 6 total = 4 FBCFs, 2 APFs)
buffer1 = np.zeros(maxDelay)
buffer2 = np.zeros(maxDelay)
buffer3 = np.zeros(maxDelay)
buffer4 = np.zeros(maxDelay)
buffer5 = np.zeros(maxDelay)
buffer6 = np.zeros(maxDelay)

# Delay and gain parameters
d1 = np.fix(0.0297 * Fs)
g1 = 0.75
d2 = np.fix(0.0371 * Fs)
g2 = -0.75
d3 = np.fix(0.0411 * Fs)
g3 = 0.75
d4 = np.fix(0.0437 * Fs)
g4 = -0.75
d5 = np.fix(0.005 * Fs)
g5 = 0.7
d6 = np.fix(0.0017 * Fs)
g6 = 0.7

# LFO parameters
rate1 = 0.6
amp1 = 8
rate2 = 0.71
amp2 = 8
rate3 = 0.83
amp3 = 8
rate4 = 0.95
amp4 = 8
rate5 = 1.07
amp5 = 8
rate6 = 1.19
amp6 = 8

# Initialize output signal
N = len(x)
out = np.zeros(N)

for n in range(N):
    # Four parallel FBCFs
    w1, buffer1 = fbcf(x[n], buffer1, Fs, n, d1, g1, amp1, rate1)
    w2, buffer2 = fbcf(x[n], buffer2, Fs, n, d2, g2, amp2, rate2)
    w3, buffer3 = fbcf(x[n], buffer3, Fs, n, d3, g3, amp3, rate3)
    w4, buffer4 = fbcf(x[n], buffer4, Fs, n, d4, g4, amp4, rate4)

    # Combine parallel paths
    combPar = 0.25 * (w1 + w2 + w3 + w4)

    # Two series all-pass filters
    w5, buffer5 = apf(combPar, buffer5, Fs, n, d5, g5, amp5, rate5)
    out[n], buffer6 = apf(w5, buffer6, Fs, n, d6, g6, amp6, rate6)

Audio(out, rate=Fs)
